# Production 추천 시스템 성능 평가

**목표**: 실제 프로덕션 코드와 동일한 로직으로 성능 평가

**평가 지표**:
- Precision@10: 상위 10개 추천 중 정답 비율
- Recall@10: 테스트 영화 중 추천된 비율
- NDCG@10: 순위를 고려한 정확도
- 추천 소요 시간

## 1단계: 환경 설정

In [1]:
import os
import sys
import time
import random
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple, Any
from dotenv import load_dotenv
from sklearn.preprocessing import MinMaxScaler
from math import log

# 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# 프로젝트 경로 설정
project_root = Path().absolute().parent.parent.parent
ai_root = project_root / 'ai'
sys.path.insert(0, str(ai_root))

# 환경 변수 로드
env_file = project_root / '.env.local'
if not env_file.exists():
    env_file = project_root / '.env'
load_dotenv(env_file)

print(f'환경 설정 완료')
print(f'  프로젝트 루트: {project_root}')
print(f'  AI 폴더: {ai_root}')
print(f'  시드: {SEED}')

환경 설정 완료
  프로젝트 루트: /Users/apple/kakao2-project/MovieSir
  AI 폴더: /Users/apple/kakao2-project/MovieSir/ai
  시드: 42


## 2단계: 추천 시스템 초기화

In [2]:
from inference.recommendation_model import HybridRecommender

# DB 설정
DB_CONFIG = {
    'host': os.getenv('DATABASE_HOST', 'localhost'),
    'port': int(os.getenv('DATABASE_PORT', 5432)),
    'database': os.getenv('DATABASE_NAME', 'moviesir'),
    'user': os.getenv('DATABASE_USER', 'movigation'),
    'password': os.getenv('DATABASE_PASSWORD', 'moviesir123')
}

# 모델 경로
ALS_MODEL_PATH = str(ai_root / 'training/als_data')
ALS_DATA_PATH = str(ai_root / 'training/als_data')

print('추천 시스템 초기화 중...')
recommender = HybridRecommender(
    db_config=DB_CONFIG,
    als_model_path=ALS_MODEL_PATH,
    als_data_path=ALS_DATA_PATH
)
print(f'\n초기화 완료')
print(f'  전체 영화 수: {len(recommender.metadata_map):,}개')
print(f'  SBERT 임베딩: {len(recommender.sbert_movie_to_idx):,}개')
print(f'  ALS 임베딩: {len(recommender.als_movie_to_idx):,}개')

추천 시스템 초기화 중...
Initializing Hybrid Recommender (SBERT + ALS, Noise-based Diversity)...
Loading metadata from database...
  Metadata loaded: 13,060 movies
Loading SBERT embeddings from database...
  SBERT movies: 13,060
Loading OTT data from database...
  OTT data loaded: 13,047 movies
  ALS movies: 11,659
Loading ALS model from /Users/apple/kakao2-project/MovieSir/ai/training/als_data
  ALS item factors shape: (65032, 128)
Pre-aligning models...
Created reverse mapping dictionary for 13,060 movies
  ⚠️  1401 movies have SBERT only (will use SBERT weight 1.0)
Pre-calculating rating scores...
Pre-calculated rating scores for 13,060 movies
Building filtering indexes...
  Year index: 105 years
  Genre index: 20 genres
  OTT index: 9 providers
Initialization complete. Target movies: 13060

초기화 완료
  전체 영화 수: 13,060개
  SBERT 임베딩: 13,060개
  ALS 임베딩: 11,659개


## 3단계: 테스트 사용자 데이터 로드

In [3]:
# ratings.csv 경로
RATINGS_CSV_PATH = ai_root / 'training/original_data/ratings.csv'

print(f'ratings.csv 로드 중...')
print(f'  파일: {RATINGS_CSV_PATH}')

df_ratings = pd.read_csv(RATINGS_CSV_PATH)
print(f'\n로드 완료')
print(f'  전체 평점 수: {len(df_ratings):,}개')
print(f'  전체 사용자: {df_ratings["userId"].nunique():,}명')
print(f'  전체 영화: {df_ratings["movieId"].nunique():,}개')

ratings.csv 로드 중...
  파일: /Users/apple/kakao2-project/MovieSir/ai/training/original_data/ratings.csv

로드 완료
  전체 평점 수: 32,000,204개
  전체 사용자: 200,948명
  전체 영화: 84,432개


In [4]:
# DB에 존재하는 영화만 필터링
valid_movie_ids = set(recommender.metadata_map.keys())
df_valid = df_ratings[df_ratings['movieId'].isin(valid_movie_ids)].copy()

print(f'DB 존재 영화 필터링:')
print(f'  필터링 전: {len(df_ratings):,}개')
print(f'  필터링 후: {len(df_valid):,}개 ({len(df_valid)/len(df_ratings)*100:.1f}%)')

DB 존재 영화 필터링:
  필터링 전: 32,000,204개
  필터링 후: 23,129,818개 (72.3%)


In [5]:
# 2000년 이후, 비성인물만 필터링 (프로덕션과 동일)
valid_filtered_ids = set()

for mid in df_valid['movieId'].unique():
    meta = recommender.metadata_map.get(mid, {})
    
    # 2000년 이상 체크
    release_date = meta.get('release_date', '')
    if release_date:
        try:
            year = int(release_date[:4])
            if year < 2000:
                continue
        except:
            continue
    
    # 비성인물 체크
    if meta.get('adult', False):
        continue
    
    valid_filtered_ids.add(mid)

df_filtered = df_valid[df_valid['movieId'].isin(valid_filtered_ids)].copy()

print(f'2000년 이후 + 비성인물 필터링:')
print(f'  필터링 후 평점: {len(df_filtered):,}개')
print(f'  필터링 후 영화: {df_filtered["movieId"].nunique():,}개')

2000년 이후 + 비성인물 필터링:
  필터링 후 평점: 8,518,542개
  필터링 후 영화: 7,951개


In [6]:
# 테스트 사용자 추출
MIN_RATINGS = 20  # 최소 평가 수
NUM_USERS = 100   # 최대 사용자 수

user_counts = df_filtered.groupby('userId').size()
valid_users = user_counts[user_counts >= MIN_RATINGS].index.tolist()
print(f'{MIN_RATINGS}개 이상 평가한 사용자: {len(valid_users):,}명')

# 상위 NUM_USERS 명 선택
top_users = user_counts.nlargest(NUM_USERS).index.tolist()

test_users = []
for user_id in top_users:
    user_ratings = df_filtered[df_filtered['userId'] == user_id].sort_values('timestamp')
    movies = user_ratings['movieId'].tolist()
    
    # 80% train, 20% test 분할
    split_idx = int(len(movies) * 0.8)
    train = movies[:split_idx]
    test = movies[split_idx:]
    
    if len(train) >= 15 and len(test) >= 5:
        test_users.append((user_id, train, test))

print(f'\n{len(test_users)}명의 테스트 사용자 추출 완료')
avg_train = sum(len(t) for _, t, _ in test_users) / len(test_users)
avg_test = sum(len(t) for _, _, t in test_users) / len(test_users)
print(f'  평균 train 영화 수: {avg_train:.1f}개')
print(f'  평균 test 영화 수: {avg_test:.1f}개')

20개 이상 평가한 사용자: 85,033명

100명의 테스트 사용자 추출 완료
  평균 train 영화 수: 1045.8개
  평균 test 영화 수: 261.9개


## 4단계: 평가 함수 정의 (프로덕션과 동일한 로직)

In [7]:
def get_top_movies_production(
    recommender,
    user_sbert_profile: np.ndarray,
    user_als_profile: np.ndarray,
    candidate_ids: List[int],
    sbert_weight: float,
    als_weight: float,
    top_k: int = 10
) -> List[Dict[str, Any]]:
    """
    프로덕션과 동일한 로직으로 상위 영화 선정
    
    수정 사항 (fianl_compare.py 대비):
    1. ALS 정규화 제거 (프로덕션은 정규화 안 함)
    2. 평점 점수 30% 반영 추가
    """
    candidate_indices = []
    for mid in candidate_ids:
        idx = recommender.movie_id_to_idx.get(mid)
        if idx is not None:
            candidate_indices.append((mid, idx))
    
    if not candidate_indices:
        return []
    
    indices = [idx for _, idx in candidate_indices]
    
    # SBERT: 정규화된 벡터 (코사인 유사도)
    sbert_similarities = recommender.target_sbert_norm[indices] @ user_sbert_profile.T
    
    # ALS: 정규화 안 함 (일반 내적) - 프로덕션과 동일!
    als_similarities = recommender.target_als_matrix[indices] @ user_als_profile.T
    
    sbert_scores = np.max(sbert_similarities, axis=1)
    als_scores = np.max(als_similarities, axis=1)
    
    scaler = MinMaxScaler()
    filtered_rating = np.array([recommender.rating_scores.get(mid, 0.0) for mid, _ in candidate_indices])
    
    if len(sbert_scores) > 1:
        norm_sbert = scaler.fit_transform(sbert_scores.reshape(-1, 1)).squeeze()
        norm_als = scaler.fit_transform(als_scores.reshape(-1, 1)).squeeze()
        norm_rating = scaler.fit_transform(filtered_rating.reshape(-1, 1)).squeeze()
    else:
        norm_sbert = sbert_scores
        norm_als = als_scores
        norm_rating = filtered_rating
    
    als_ids = set(recommender.als_movie_to_idx.keys())
    
    movie_scores = []
    for i, (mid, _) in enumerate(candidate_indices):
        if mid in als_ids:
            model_score = sbert_weight * norm_sbert[i] + als_weight * norm_als[i]
        else:
            model_score = norm_sbert[i]
        
        rating_score = norm_rating[i] if isinstance(norm_rating, np.ndarray) else norm_rating
        final_score = model_score * 0.7 + rating_score * 0.3
        
        meta = recommender.metadata_map.get(mid, {})
        movie_scores.append({
            'movie_id': mid,
            'title': meta.get('title', 'Unknown'),
            'genres': meta.get('genres', []),
            'score': final_score
        })
    
    movie_scores.sort(key=lambda x: x['score'], reverse=True)
    return movie_scores[:top_k]

print('평가 함수 정의 완료 (프로덕션 로직 반영)')
print('  - ALS: 정규화 없음 (일반 내적)')
print('  - 평점 점수: 30% 반영')

평가 함수 정의 완료 (프로덕션 로직 반영)
  - ALS: 정규화 없음 (일반 내적)
  - 평점 점수: 30% 반영


In [8]:
def calculate_precision_at_k(recommendations, ground_truth, k=10):
    top_k = recommendations[:k]
    relevant = set(ground_truth)
    hits = len(set(top_k) & relevant)
    return hits / k if k > 0 else 0.0

def calculate_recall_at_k(recommendations, ground_truth, k=10):
    top_k = recommendations[:k]
    relevant = set(ground_truth)
    hits = len(set(top_k) & relevant)
    return hits / len(relevant) if len(relevant) > 0 else 0.0

def calculate_ndcg_at_k(recommendations, ground_truth, k=10):
    top_k = recommendations[:k]
    relevant = set(ground_truth)
    relevance = np.array([1.0 if mid in relevant else 0.0 for mid in top_k])
    
    if relevance.sum() == 0:
        return 0.0
    
    dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevance))
    ideal_relevance = np.sort(relevance)[::-1]
    idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevance))
    
    return dcg / idcg if idcg > 0 else 0.0

print('평가 지표 함수 정의 완료')

평가 지표 함수 정의 완료


## 5단계: 평가 실행

In [9]:
K = 10
SBERT_WEIGHT = 0.7
ALS_WEIGHT = 0.3

precision_scores = []
recall_scores = []
ndcg_scores = []
elapsed_times = []

print('='*60)
print('Production 추천 시스템 평가 시작')
print(f'  - K: {K}')
print(f'  - SBERT 가중치: {SBERT_WEIGHT}')
print(f'  - ALS 가중치: {ALS_WEIGHT}')
print(f'  - 테스트 사용자: {len(test_users)}명')
print('='*60)

for idx, (user_id, train_movies, test_movies) in enumerate(test_users):
    start_time = time.time()
    
    try:
        user_sbert_profile, user_als_profile = recommender._get_user_profile(train_movies)
        all_movie_ids = list(recommender.metadata_map.keys())
        candidate_ids = [mid for mid in all_movie_ids if mid not in train_movies]
        
        top_movies = get_top_movies_production(
            recommender=recommender,
            user_sbert_profile=user_sbert_profile,
            user_als_profile=user_als_profile,
            candidate_ids=candidate_ids,
            sbert_weight=SBERT_WEIGHT,
            als_weight=ALS_WEIGHT,
            top_k=K
        )
        
        elapsed = time.time() - start_time
        elapsed_times.append(elapsed)
        
        top_k_ids = [m['movie_id'] for m in top_movies]
        
        precision_scores.append(calculate_precision_at_k(top_k_ids, test_movies, K))
        recall_scores.append(calculate_recall_at_k(top_k_ids, test_movies, K))
        ndcg_scores.append(calculate_ndcg_at_k(top_k_ids, test_movies, K))
        
        if (idx + 1) % 20 == 0:
            print(f'  진행: {idx + 1}/{len(test_users)} users')
            
    except Exception as e:
        print(f'  User {user_id} 평가 실패: {e}')
        continue

print(f'\n평가 완료: {len(precision_scores)}명')

Production 추천 시스템 평가 시작
  - K: 10
  - SBERT 가중치: 0.7
  - ALS 가중치: 0.3
  - 테스트 사용자: 100명
  진행: 20/100 users
  진행: 40/100 users
  진행: 60/100 users
  진행: 80/100 users
  진행: 100/100 users

평가 완료: 100명


## 6단계: 결과 분석

In [10]:
results = {
    'Precision@10': np.mean(precision_scores),
    'Recall@10': np.mean(recall_scores),
    'NDCG@10': np.mean(ndcg_scores),
    'avg_time': np.mean(elapsed_times),
    'std_time': np.std(elapsed_times),
    'num_users': len(precision_scores)
}

print('='*60)
print('Production 추천 시스템 성능 평가 결과')
print('='*60)
print(f'\n정확도 지표:')
print(f'  Precision@{K}: {results["Precision@10"]:.4f}')
print(f'  Recall@{K}:    {results["Recall@10"]:.4f}')
print(f'  NDCG@{K}:      {results["NDCG@10"]:.4f}')
print(f'\n속도 지표:')
print(f'  평균 추천 시간: {results["avg_time"]:.3f}s (+/-{results["std_time"]:.3f}s)')
print(f'\n평가 규모:')
print(f'  평가 사용자: {results["num_users"]}명')
print('='*60)

Production 추천 시스템 성능 평가 결과

정확도 지표:
  Precision@10: 0.2110
  Recall@10:    0.0085
  NDCG@10:      0.4657

속도 지표:
  평균 추천 시간: 0.387s (+/-0.124s)

평가 규모:
  평가 사용자: 100명


In [11]:
print('점수 분포 분석:')
print(f'\nPrecision@{K}:')
print(f'  Min: {np.min(precision_scores):.4f}')
print(f'  Max: {np.max(precision_scores):.4f}')
print(f'  Median: {np.median(precision_scores):.4f}')
print(f'  Std: {np.std(precision_scores):.4f}')

print(f'\nRecall@{K}:')
print(f'  Min: {np.min(recall_scores):.4f}')
print(f'  Max: {np.max(recall_scores):.4f}')
print(f'  Median: {np.median(recall_scores):.4f}')
print(f'  Std: {np.std(recall_scores):.4f}')

print(f'\nNDCG@{K}:')
print(f'  Min: {np.min(ndcg_scores):.4f}')
print(f'  Max: {np.max(ndcg_scores):.4f}')
print(f'  Median: {np.median(ndcg_scores):.4f}')
print(f'  Std: {np.std(ndcg_scores):.4f}')

점수 분포 분석:

Precision@10:
  Min: 0.0000
  Max: 0.7000
  Median: 0.2000
  Std: 0.1536

Recall@10:
  Min: 0.0000
  Max: 0.0314
  Median: 0.0081
  Std: 0.0066

NDCG@10:
  Min: 0.0000
  Max: 0.9060
  Median: 0.4842
  Std: 0.2524


In [12]:
summary_df = pd.DataFrame({
    '지표': ['Precision@10', 'Recall@10', 'NDCG@10', '평균 추천 시간(s)'],
    '값': [
        f"{results['Precision@10']:.4f}",
        f"{results['Recall@10']:.4f}",
        f"{results['NDCG@10']:.4f}",
        f"{results['avg_time']:.3f}"
    ]
})

print('\n결과 요약 테이블:')
print(summary_df.to_string(index=False))


결과 요약 테이블:
          지표      값
Precision@10 0.2110
   Recall@10 0.0085
     NDCG@10 0.4657
 평균 추천 시간(s)  0.387


## 7단계: 결과 저장

In [13]:
import json
from datetime import datetime

output_data = {
    'metadata': {
        'created_at': datetime.now().isoformat(),
        'model': 'Production Hybrid (SBERT 0.7 + ALS 0.3)',
        'k': K,
        'num_users': len(precision_scores),
        'seed': SEED
    },
    'results': {
        'precision@10': float(results['Precision@10']),
        'recall@10': float(results['Recall@10']),
        'ndcg@10': float(results['NDCG@10']),
        'avg_time': float(results['avg_time']),
        'std_time': float(results['std_time'])
    }
}

output_path = Path('production_evaluation_results.json')
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f'결과 저장 완료: {output_path}')

결과 저장 완료: production_evaluation_results.json
